In [61]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
from scipy.stats import skew
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

# Numerical Data only test

### Import the training dataset and Feature engineering

In [62]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
print(f'Train shape : {train.shape}')
print(f'Test shape : {test.shape}')

Train shape : (1460, 81)
Test shape : (1459, 80)


In [63]:
# Bring train and test together for pre processing and featur engineering

data = pd.concat([train,test], axis=0)
y_train = train['SalePrice']
data = data.drop(['Id', 'SalePrice'], axis=1)
print(data.shape)

(2919, 79)


In [64]:
#Columns containing most null values
total = data.isnull().sum().sort_values(ascending=False)
percent = (data.isnull().sum() / data.isnull().count()*100).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
print(missing_data.head(10))

              Total    Percent
PoolQC         2909  99.657417
MiscFeature    2814  96.402878
Alley          2721  93.216855
Fence          2348  80.438506
FireplaceQu    1420  48.646797
LotFrontage     486  16.649538
GarageYrBlt     159   5.447071
GarageFinish    159   5.447071
GarageQual      159   5.447071
GarageCond      159   5.447071


In [65]:
#Dropping columns with > 5 null values
data.drop((missing_data[missing_data['Total'] > 5]).index, axis=1, inplace=True)
#Sorting columns w.r.t null values
total = data.isnull().sum().sort_values(ascending=False)
total.head(20)

MSZoning        4
Functional      2
BsmtFullBath    2
BsmtHalfBath    2
Utilities       2
BsmtFinSF2      1
Exterior2nd     1
GarageCars      1
GarageArea      1
BsmtFinSF1      1
BsmtUnfSF       1
Exterior1st     1
TotalBsmtSF     1
Electrical      1
SaleType        1
KitchenQual     1
HalfBath        0
FullBath        0
BedroomAbvGr    0
KitchenAbvGr    0
dtype: int64

In [66]:
#Filling the numeric data
numeric_missed = ['BsmtFinSF1',
'BsmtFinSF2',
'BsmtUnfSF',
'TotalBsmtSF',
'BsmtFullBath',
'BsmtHalfBath',
'GarageArea',
'GarageCars']
for feature in numeric_missed:
    #data[feature] = data[feature].fillna(0)
    data[feature] = data[feature].fillna(data[feature].mean())
#Filling the categorical data
categorical_missed = ['Exterior1st',
'Exterior2nd',
'SaleType',
'MSZoning',
'Electrical',
'KitchenQual',
'Functional']
for feature in categorical_missed:
    data[feature] = data[feature].fillna(data[feature].mode()[0])
#Deleting 'Utilities' column
data.drop(['Utilities'], axis=1, inplace=True)


In [67]:
#Top skewed columns
numeric_features = data.dtypes[data.dtypes != 'object'].index
skewed_features = data[numeric_features].apply(lambda x: skew(x)).sort_values(ascending=False)
high_skew = skewed_features[abs(skewed_features) > 0.5]
print(high_skew)

MiscVal          21.947195
PoolArea         16.898328
LotArea          12.822431
LowQualFinSF     12.088761
3SsnPorch        11.376065
KitchenAbvGr      4.302254
BsmtFinSF2        4.146034
EnclosedPorch     4.003891
ScreenPorch       3.946694
BsmtHalfBath      3.931343
OpenPorchSF       2.535114
WoodDeckSF        1.842433
1stFlrSF          1.469604
BsmtFinSF1        1.425233
MSSubClass        1.375457
GrLivArea         1.269358
TotalBsmtSF       1.162484
BsmtUnfSF         0.919508
2ndFlrSF          0.861675
TotRmsAbvGrd      0.758367
Fireplaces        0.733495
HalfBath          0.694566
BsmtFullBath      0.623955
OverallCond       0.570312
YearBuilt        -0.599806
dtype: float64


In [68]:
#Transforming skewed columns
for feature in high_skew.index:
    data[feature] = np.log1p(data[feature])

In [69]:
#Converting categorical data to numerical
data = pd.get_dummies(data)
data.head()

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,4.110874,9.042040,7,1.791759,7.602900,2003,6.561031,0.0,5.017280,6.753438,...,0,0,0,1,0,0,0,0,1,0
1,3.044522,9.169623,6,2.197225,7.589336,1976,6.886532,0.0,5.652489,7.141245,...,0,0,0,1,0,0,0,0,1,0
2,4.110874,9.328212,7,1.791759,7.601902,2002,6.188264,0.0,6.075346,6.825460,...,0,0,0,1,0,0,0,0,1,0
3,4.262680,9.164401,7,1.791759,7.557995,1970,5.379897,0.0,6.293419,6.629363,...,0,0,0,1,1,0,0,0,0,0
4,4.110874,9.565284,8,1.791759,7.601402,2000,6.486161,0.0,6.196444,7.044033,...,0,0,0,1,0,0,0,0,1,0


In [70]:
#Dividing data back into train & test
train =data[:len(y_train)]
test = data[len(y_train):]
#Printing thier shapes
print(train.shape, test.shape)

(1460, 218) (1459, 218)


In [71]:
# [CHECKPOINT 5][5 points]
def  feature_normalization(X):
    """
    Normalizes the features in X. returns a normalized version of X where
    the mean value of each feature is 0 and the standard deviation
    is 1. This is often a good preprocessing step to do when working with
    learning algorithms.
    
    Parameters
    ----------
    X : array_like
        The dataset of shape (m x n).
    
    Returns
    -------
    X_norm : array_like
        The normalized dataset of shape (m x n).
    
    Instructions
    ------------
    First, for each feature dimension, compute the mean of the feature
    and subtract it from the dataset, storing the mean value in mu. 
    Next, compute the  standard deviation of each feature and divide
    each feature by it's standard deviation, storing the standard deviation 
    in sigma. 
    
    Note that X is a matrix where each column is a feature and each row is
    an example. You needto perform the normalization separately for each feature. 
    
    Hint
    ----
    You might find the 'np.mean' and 'np.std' functions useful.
    """
    mu = []
    sigma = []
    X_norm = np.copy(X)
    
    col_to_remove = []
    for i in range(X_norm.shape[1]):
        mu.append(np.mean(X_norm[:,i]))
        sigma.append(np.std(X_norm[:,i]))
    X_norm = np.delete(X_norm, col_to_remove, axis=1)

    # =========================== YOUR CODE HERE =====================
    for i in range(X_norm.shape[1]):
        if np.std(X_norm[:,i]) != 0:
            X_norm[:,i] = (X_norm[:,i] - np.mean(X_norm[:,i])) / np.std(X_norm[:,i])
        else:
            X_norm[:,i] = (X_norm[:,i] - np.mean(X_norm[:,i])) / 0.01
    
    
    # ================================================================
    return X_norm, mu, sigma

In [72]:
normal_data, mu, sigma = feature_normalization(train)
# Add intercept Column
normal_data = np.insert(normal_data, 0, np.ones(normal_data.shape[0]), axis=1)
#normal_data[:,0] = np.ones(data.shape[0])
pd.DataFrame(normal_data)

,0,1,2,3,4,5,6,7,8,9,...,209,210,211,212,213,214,215,216,217,218
0,1.0,0.424462,-0.133270,0.651479,-0.460408,1.045180,0.878668,0.779431,-0.355342,-0.340511,...,-0.058621,-0.301962,-0.045376,0.390293,-0.272616,-0.052414,-0.091035,-0.117851,0.467651,-0.305995
1,1.0,-1.125202,0.113413,-0.071836,1.948163,0.163445,-0.429577,0.888257,-0.355342,0.002218,...,-0.058621,-0.301962,-0.045376,0.390293,-0.272616,-0.052414,-0.091035,-0.117851,0.467651,-0.305995
2,1.0,0.424462,0.420049,0.651479,-0.460408,0.980275,0.830215,0.654803,-0.355342,0.230372,...,-0.058621,-0.301962,-0.045376,0.390293,-0.272616,-0.052414,-0.091035,-0.117851,0.467651,-0.305995
3,1.0,0.645073,0.103317,0.651479,-0.460408,-1.873790,-0.720298,0.384539,-0.355342,0.348034,...,-0.058621,-0.301962,-0.045376,0.390293,3.668167,-0.052414,-0.091035,-0.117851,-2.138345,-0.305995
4,1.0,0.424462,0.878431,1.374795,-0.460408,0.947798,0.733308,0.754400,-0.355342,0.295711,...,-0.058621,-0.301962,-0.045376,0.390293,-0.272616,-0.052414,-0.091035,-0.117851,0.467651,-0.305995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1.0,0.424462,-0.259231,-0.071836,-0.460408,0.915305,0.733308,-1.414140,-0.355342,0.654093,...,-0.058621,-0.301962,-0.045376,0.390293,-0.272616,-0.052414,-0.091035,-0.117851,0.467651,-0.305995
1456,1.0,-1.125202,0.725429,-0.071836,0.455288,0.229170,0.151865,0.816966,2.409693,0.394815,...,-0.058621,-0.301962,-0.045376,0.390293,-0.272616,-0.052414,-0.091035,-0.117851,0.467651,-0.305995
1457,1.0,0.645073,-0.002359,0.651479,2.574033,-0.997641,1.024029,0.464947,-0.355342,0.609301,...,-0.058621,-0.301962,-0.045376,0.390293,-0.272616,-0.052414,-0.091035,-0.117851,0.467651,-0.305995
1458,1.0,-1.125202,0.136833,-0.795151,0.455288,-0.697090,0.539493,-0.106220,3.405917,-3.047600,...,-0.058621,-0.301962,-0.045376,0.390293,-0.272616,-0.052414,-0.091035,-0.117851,0.467651,-0.305995


### Normal Equation and Polynomial Regression Functions

### 3.2 Normal equation

We have learned that the closed-form solution to linear regression is

$$ \theta = \left( X^T X\right)^{-1} X^T\vec{y}$$

Using this formula does not require any feature scaling, and you will get an exact solution in one calculation: there is no “loop until convergence” like in gradient descent. 

Hint: Before implement normal_equation function, following example shows how to convert array data to a matrix with a shape of $m \times 1$, and generate polynomial features matrix $[1, X, X^2, X^3, \cdots]$. 

In [73]:
# [CHECKPOINT 2][10 points]
def normal_equation(X, Y, n):
    """
    Computes the closed-form solution to linear regression using the normal equations.
    
    Parameters
    ----------
    X : array_like
        The dataset of shape (m, ).
    
    Y : array_like
        The value at each data point. A vector of shape (m, ).
        
    n : the order of polynomial regression model
        Remember the number of features will be n+1.
    
    Returns
    -------
    theta : array_like
        Estimated polynomial regression parameters. A vector of shape (n+1, ).
    
    Instructions
    ------------
    Complete the code to compute the closed form solution to linear
    regression and put the result in theta.
    
    Hint
    ----
    Look up the function `np.linalg.pinv` for computing matrix inverse.
    """
    #print("Normal equation start")
    #print("n: ", n)
    theta = np.zeros(n+1)
    #print("theta: ", theta)
    
    # ===================== YOUR CODE HERE ============================
    X_t = np.transpose(X)
    theta = np.matmul(X_t, X)
    theta = np.linalg.pinv(theta)
    tmp = np.matmul(X_t, Y)
    theta = np.matmul(theta, tmp)
    # =================================================================
    return theta.flatten()

In [74]:
# [CHECKPOINT 3][5 points]
def polynomial_deploy(X, theta):
    """
    Computes the polynomial regression prediction for data X.
    
    Parameters
    ----------
    X : array_like
        The input data. A vector of shape (m, ).
    
    theta : array_like
        Polynomial regression parameters. A vector of shape (n+1, ).
    
    Returns
    -------
    Y : array_like
        Polynomial prediction. A vector of shape (m, ).
    
    """
    #print("Begin polynomial deploy")
    n = theta.size
    Y = np.array([])
    
    # ===================== YOUR CODE HERE ============================

    theta_t = theta.reshape(n, 1)

    #Y = np.matmul(np.transpose(theta), X)
    Y = np.matmul(X, theta_t)
            
    #print("X poly size: ", X_poly.shape)
    
    # ===================== YOUR CODE HERE ============================
    return Y.flatten()

In [75]:
#MSE helper func
def MSE_func(m, hypothesis, actual):
    MSE = 0
    for i in range(m):
            MSE += (hypothesis[i] - actual[i]) ** 2
    return MSE

In [76]:
# Split Datasets for testing and training
x_train, x_test, y_train, y_test = train_test_split(normal_data, y_train, test_size=0.2)
x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [77]:
# Predict func
def predict(x, y):
    theta = normal_equation(x, y, x.shape[1])
    prediction = polynomial_deploy(x,  theta)
    RMSE = metrics.mean_squared_error(y, prediction,  squared=False)
    VarScore = metrics.explained_variance_score(y,prediction)
    return RMSE, VarScore

### Polynomial Function Predictions and MSE

In [78]:
'''from sklearn import metrics

n = data.shape[1]
m = data.shape[0]
#print("n:", n)
X_data = data
theta = normal_equation(X_data, y, n)
#print("theta x set: ", theta)
prediction = polynomial_deploy(X_data, theta)
#print(prediction.shape)
MSE = MSE_func(X_data.shape[0], prediction, y)
print("MSE:", MSE)
print('VarScore:',metrics.explained_variance_score(y,prediction))
print("RMSE:", metrics.mean_squared_error(y,prediction))'''

'from sklearn import metrics\n\nn = data.shape[1]\nm = data.shape[0]\n#print("n:", n)\nX_data = data\ntheta = normal_equation(X_data, y, n)\n#print("theta x set: ", theta)\nprediction = polynomial_deploy(X_data, theta)\n#print(prediction.shape)\nMSE = MSE_func(X_data.shape[0], prediction, y)\nprint("MSE:", MSE)\nprint(\'VarScore:\',metrics.explained_variance_score(y,prediction))\nprint("RMSE:", metrics.mean_squared_error(y,prediction))'

In [79]:
#training
RMSE, VarScore= predict(x_train, y_train)
print("RMSE: ", RMSE)
print("Var Score: ",  VarScore)

RMSE:  24971.94439939039
Var Score:  0.9009595839572639


In [80]:
#testing
RMSE, VarScore= predict(x_test, y_test)
print("RMSE: ", RMSE)
print("Var Score: ",  VarScore)

RMSE:  17081.514224782208
Var Score:  0.954038644233175
